In [ ]:



import datav3 as mydataloader



In [2]:
import tensorflow as tf
from tensorflow.contrib import rnn

import numpy as np
import os

unhappy_directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data_test/data/0/'
unhappy_directory = os.fsencode(unhappy_directory_path)

happy_directory_path='/media/saurabh/New Volume/tf_audio_sentiment/data_test/data/1/'
happy_directory = os.fsencode(happy_directory_path)

model_path = "/media/saurabh/New Volume/tf_audio_sentiment/models/rnn_model.ckpt"

In [3]:
# Parameters
learning_rate = 0.0001
training_epochs = 25
batch_size = 2
display_step = 1

In [4]:
timesteps=125 # max timesteps
num_input = 22 # size of vector, at each timestep
num_classes = 2 # happy or sad

num_hidden = 200 # hidden layer num of features

x = tf.placeholder(tf.float32, [None,timesteps, num_input]) 
y = tf.placeholder(tf.int32, [None]) #  Index of output , 0 = sad , 1 = happy

In [5]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]
        

In [6]:
# Define weights
weights = {
    # Hidden layer weights => 2*n_hidden because of forward + backward cells
    'out': tf.Variable(tf.random_normal([2*num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}


In [7]:
def BiRNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x, timesteps, 1)

    # Define lstm cells with tensorflow
    # Forward direction cell
    lstm_fw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    # Backward direction cell
    lstm_bw_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    try:
        outputs, _, _ = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                              dtype=tf.float32)
    except Exception: # Old TensorFlow version only returns outputs not states
        outputs = rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x,
                                        dtype=tf.float32)
        
    # Linear activation, using rnn inner loop last output
    #return tf.matmul(outputs[-1], weights['out']) + biases['out']
    return outputs[-1]

In [8]:
Logits = BiRNN(x, weights, biases)
prediction = tf.nn.softmax(Logits) # use only for inference, softmax included in loss function


In [9]:
# Minimize error using cross entropy
#cost = tf.reduce_mean(-tf.reduce_sum(y_one_hot*tf.log(pred), reduction_indices=1))

cost = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=Logits)

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [10]:
# 'Saver' op to save and restore all the variables
saver = tf.train.Saver()

In [11]:
next_element = mydataloader.iterator.get_next()

In [13]:
# Start training
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        
        
        sess.run(mydataloader.iterator.initializer)
        while True:
            try:
                Next_element=sess.run(next_element)
               # print(Next_element)
              # train single "unhappy" example
                _, cost = sess.run([optimizer, cost], feed_dict={x: Next_element[0],
                                                          y: Next_element[1]})
            
                 
                print(cost)   
              
            except tf.errors.OutOfRangeError:
              break
                
                
                 
            
                 
        
          
         # Save model weights to disk
     #   save_path = saver.save(sess, model_path)
     #   print("Model saved in file: %s" % save_path)
        
        
         
    
   

  

21_10_48022B.wav
21_15_40334B.wav
21_23_56047A.wav
[ 6.00690937  6.00392866  5.98582745]
21_10_48166B.wav
21_22_48035A.wav
21_15_40082B.wav


TypeError: Fetch argument array([ 6.00690937,  6.00392866,  5.98582745], dtype=float32) has invalid type <class 'numpy.ndarray'>, must be a string or Tensor. (Can not convert a ndarray into a Tensor or Operation.)